In [1]:
#imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
import sys
pd.set_option('display.max_colwidth', None)

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

In [2]:
books_df = pd.read_csv('./data/books.csv',usecols=['customer_id', 'product_id', 'product_title', 'star_rating', 'review_date'])
books_df.head() #preview

,customer_id,product_id,product_title,star_rating,review_date
0,12076615,0385730586,Sisterhood of the Traveling Pants (Book 1),4.0,2005-10-14
1,12703090,0811828964,The Bad Girl's Guide to Getting What You Want,3.0,2005-10-14
2,31048862,0316769487,The Catcher in the Rye,4.0,2005-10-14
3,53000124,0805076069,Bait and Switch: The (Futile) Pursuit of the American Dream,4.0,2005-10-14
4,15280864,0446531081,Hour Game (King & Maxwell),4.0,2005-10-14


In [4]:
sys.getsizeof(books_df)/1_000_000_000

0.370858643

In [5]:
books_df.shape

(1489354, 5)

In [6]:
books_recommender = ut.make_recommender_df(books_df)
books_recommender.head()

Dropping 42988 duplicate values.
Unique customers: 795389
Unique products: 49547
Size of matrix: (46575, 795389)
Making df dictionary...
Dictionary made - sparse dataframe under construction...
Size of Recommender df: 0.422205132 GB


,"Effective VisualAge(r) for Java, Version 3: Includes Coverage of Versions 3.02 and 3.5",Polkabats and Octopus Slacks: 14 Stories,Building Harlequin's Moon,BABY ER : The Heroic Doctors and Nurses Who Perform Medicine's Tiniest Miracles,"Data Mining: Technologies, Techniques, Tools, and Trends",To Seek Out New Life: The Biology of Star Trek,Special Edition Using Filemaker Pro 5,Drinker of Blood (Lord Meren Mysteries),"Hormones, Health, and Happiness: A Natural Medical Formula for Rediscovering Youth",The Machinery of Freedom: Guide to a Radical Capitalism,...,The Global Citizen: A Guide to Creating an International Life and Career,"Winning Without Intimidation : How to Master the Art of Positive Persuasion in Today's Real World in Order to Get What You Want, When You Want It",Father Like A Tree,Rethinking the Sales Force: Redefining Selling to Create and Capture Customer Value,The Mysteries Within: A Surgeon Reflects on Medical Myths,"Now, Discover Your Strengths",The Safari Companion: A Guide to Watching African Mammals,The Rabbit Handbook,RollerCoaster Tycoon: Corkscrew Follies (Prima's Official Strategy Guide),"Beyond Ender's Game: Speaker for the Dead, Xenocide and Children of the Mind (Box Set)"
"Effective VisualAge(r) for Java, Version 3: Includes Coverage of Versions 3.02 and 3.5",0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Polkabats and Octopus Slacks: 14 Stories,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Building Harlequin's Moon,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BABY ER : The Heroic Doctors and Nurses Who Perform Medicine's Tiniest Miracles,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"Data Mining: Technologies, Techniques, Tools, and Trends",1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
books_recommender.shape

(46575, 46575)

In [8]:
books_recommender.to_pickle('./pickles/books_recommender.pkl')

In [9]:
movies_test = pd.read_pickle('./pickles/movie_recommender.pkl')
print(movies_test.shape)
movies_test.head()

(72385, 72385)


,National Lampoon's Pucked,MARIZA,Monarchy Complete Collection,Goodbye Charlie,Cast a Giant Shadow [Blu-ray],Dragon Ball Z: Kai: Part Two [Blu-ray],"Shepherding a Child's Heart, Tedd Tripp - DVD Set",A Christmas Story (Ultimate Collector's Edition),Battlestar Galactica: Season 3 [Blu-ray],Grey's Anatomy - Season 1 DVD (2005),...,United 93 (Widescreen Edition),Psych: The Complete Third Season,Magical Shopping Arcade Abenobashi Vols. 1-3 (Boxed Set),Lady Caroline Lamb,Sundown (artisan),Editing How-to Videos with Sony Vegas Movie Studio,Road to Morocco,America - In Concert,Little Britain: The Complete Collection,Anchorman 2: The Legend Continues
National Lampoon's Pucked,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.991978,1.0,1.0,0.989356,1.0,1.0,1.0,1.0,1.00000
MARIZA,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.00000
Monarchy Complete Collection,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.00000
Goodbye Charlie,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,0.97466
Cast a Giant Shadow [Blu-ray],1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.00000


In [10]:
movies_test.dtypes

National Lampoon's Pucked                             Sparse[float64, 1]
MARIZA                                                Sparse[float64, 1]
Monarchy Complete Collection                          Sparse[float64, 1]
Goodbye Charlie                                       Sparse[float64, 1]
Cast a Giant Shadow [Blu-ray]                         Sparse[float64, 1]
                                                             ...        
Editing How-to Videos with Sony Vegas Movie Studio    Sparse[float64, 1]
Road to Morocco                                       Sparse[float64, 1]
America - In Concert                                  Sparse[float64, 1]
Little Britain: The Complete Collection               Sparse[float64, 1]
Anchorman 2: The Legend Continues                     Sparse[float64, 1]
Length: 72385, dtype: object